In [1]:
import spacy
import json
import os
from nltk.corpus import stopwords, words
import pandas as pd
import gensim
import gensim.corpora as corpora
import re
import contractions
import string

In [455]:
files = []
subdir = tuple(os.walk("../MergedData/merged_movie_jsons/2023"))
for file_name in tuple(subdir)[0][-1]:
    with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
        files.append(json.load(f))

In [456]:
corpus = []
my_pos = set(["ADJ"])
nlp = spacy.load("en_core_web_lg") #python -m spacy download en_core_web_lg

# [print(file["title"]) for file in files]
for file in files:
    if file["title"] != "Super Mario Bros.":
         continue
    content = [review["content"] for review in file["reviews"]]
    for review in content:
        s = nlp(review)
        clean = set()
        for word in s:
            # if word.pos_ in my_pos:
                clean.add(str(word.lemma_))
        corpus.append(" ".join(clean))
    break

In [457]:
# corpus
# remove non english words again
# remove numbers
# remove words shorted than 3 letters
# remove punctuations
# print(type(stopwords.words("english")))

In [458]:
with open("../stopwords.json", "r") as f:
    common = json.load(f)
    common.extend(stopwords.words("english"))
    common = set(common)


eng_words = set(words.words())

def clean(text: str):
    
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)

    text = re.sub("[^a-zA-Z0-9 ]+", " ", text)

    # remove emails
    text = re.sub("[\w\.-]+@[\w\.-]+\.\w+", " ", text)
    
    text = " ".join([word for word in text.split() if word not in common and word in eng_words and len(word) > 2])

     # remove multispaces
    text = re.sub('\s+', " ", text)

    # lemmatizer.lemmatize()
    return text.strip()

In [459]:
proc_data = [clean(movie_revs).split() for movie_revs in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [460]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict,  normalize=True, dictionary=input_dict, slope=2.5, smartirs="Lfc")
vector = model[input_corpus]

In [461]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f
# d

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gensim/models/tfidfmodel.py:180: RuntimeWarning: Mean of empty slice.
  return (1 + np.log2(tf)) / (1 + np.log2(tf.mean(axis=0)))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [462]:
a = {x.lemma_:v for k, v in d.items() for x in nlp(k)if x.pos_ in my_pos}

In [463]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [464]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df["TF-IDF"].between(0.8 * df["TF-IDF"].median(), 1.05 * df["TF-IDF"].median(), inclusive=True))
    # pass

In [465]:
# kmeans = KMeans(n_clusters=100, init="k-means++", n_init=10000)
# vect = TfidfVectorizer(stop_words="english")
# features = vect.fit_transform([ " ".join(data) for data in proc_data])
# kmeans.fit(features)

In [466]:
# df2 = pd.DataFrame(corpus)
# df2["clusters"] = kmeans.labels_
# df2

In [467]:
# print("Cluster centrojds: \n")
# order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
# terms = vect.get_feature_names_out()
# for i in range (100):
#     print("cluster %d:" % i)
#     for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
#         print(' %s ' % terms [j])

In [468]:
df = pd.DataFrame(a.values(), index=a.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df["TF-IDF"])
#     pass

fabulous           1.000
fateful            0.980
relatable          0.969
enjoyable          0.839
appropriate        0.833
violent            0.821
sensational        0.815
nonstop            0.774
autistic           0.759
generational       0.747
disappointing      0.741
tough              0.738
vivid              0.722
cinematic          0.698
playful            0.679
adorable           0.646
controversial      0.637
senior             0.624
uncomfortable      0.614
tolerable          0.613
despicable         0.592
american           0.584
meaningful         0.584
occasional         0.575
lame               0.574
active             0.573
invincible         0.570
nice               0.562
irrelevant         0.557
authentic          0.549
wholesome          0.548
vibrant            0.547
stereotypical      0.545
heroic             0.543
vintage            0.534
superb             0.532
unexpected         0.529
definite           0.529
uncanny            0.528
bare               0.524


In [472]:


med = df["TF-IDF"].quantile(.5)
mean = df["TF-IDF"].mean()
vibes = set()
# mean = df["TF-IDF"].quantile(.9)
# a = {k: v for k, v in sorted(a.items(), key=lambda item: item[1], reverse=True)}
for k, v in a.items():
    if abs(med - v) <= 0.05:
        vibes.add(k)
        print(k)
# vibes
# print(file["title"])

nostalgic
equivalent
undeserved
simplistic
thin
colorful
high
captivating
casual
wonderful
dear
surprising
sweet
cheap
mid
subjective
silly
available
flawless
negative
primary
faithful
creative
refreshing
flimsy
pleasant
spectacular
clever
predictable
thrilling
uninspired
respectful
loud
outstanding
expressive
silent
believable
consistent
worried
wise
dimensional
abstract
previous
psychotic
understandable
pro
subject
unnecessary
enormous
progressive
annoying
sophisticated
confident
emotional
mild
random
accurate
stale
unacceptable
phenomenal
aggressive
cruel
nasty
ugly
proper
quirky
cappy
elderly
introductory
illogical
thick
playable
safe
magical
amusing
humorous
uncontainable
sarcastic
rich
serviceable
astounding
barren
trivial
healthy
distinctive
apparent
remarkable
concise
successful
eternal
german
instrumental
exceptional
extraordinary
comfortable
emotive
attentive
transparent
genuine
frequent


In [473]:
# cnt = {}
# for sent in corpus:
#     sent = clean(sent)
#     for word in set(sent.split()):
#         cnt[word] = cnt.get(word, 0) + 1

# cnt = {k: v for k, v in sorted(cnt.items(), key=lambda item: item[1], reverse=True)}

# for c in cnt:
#     print(c, cnt[c])

movie 805
love 339
Mario 304
game 298
fun 207
see 206
like 187
watch 175
family 154
fan 146
time 132
really 131
enjoy 128
well 123
story 117
reference 111
make 109
character 109
would 108
film 105
video 102
one 99
lot 88
play 86
animation 84
think 81
get 79
year 79
critic 72
egg 69
awesome 67
adult 65
nostalgia 65
take 65
recommend 64
plot 63
music 63
definitely 62
feel 61
even 60
also 59
could 59
super 58
whole 57
expect 57
give 56
back 56
say 56
want 55
every 54
nostalgic 52
entertaining 51
voice 51
ever 50
bring 50
cute 50
age 49
grow 48
action 48
way 48
thing 47
must 47
everything 44
Easter 43
overall 41
enjoyable 38
know 38
child 38
absolutely 38
audience 38
childhood 38
long 38
scene 37
visual 37
look 36
wait 36
never 36
start 36
laugh 36
hope 34
experience 33
people 33
song 33
animate 33
worth 32
cast 32
Luigi 32
keep 32
theater 31
pretty 31
sequel 31
franchise 31
end 31
especially 30
memory 30
easter 29
though 29
still 28
sure 28
illumination 28
peach 28
anyone 27
son 27
nice 2